# Wine Quality Prediction
Este notebook demonstra como treinar uma rede neural MLP para prever a qualidade de vinhos.

## 1. Carregamento dos Dados
Aqui carregamos os CSVs e unimos os dados de vinho tinto e branco.

In [ ]:
import pandas as pd
red = pd.read_csv('winequality-red.csv', sep=';')
white = pd.read_csv('winequality-white.csv', sep=';')
data = pd.concat([red, white], ignore_index=True)
data.head()

### Conceitos
- **Aprendizado supervisionado**: usamos exemplos rotulados (qualidade) para treinar o modelo.
- **Classificação multiclasse**: a variável alvo possui três classes (baixa, média e alta qualidade).

## 2. Pré-processamento

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Transformar a qualidade em classes 0, 1, 2
data['quality'] = data['quality'].apply(lambda q: 0 if q <=5 else (1 if q==6 else 2))

X = data.drop('quality', axis=1).values
y = data['quality'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

### Conceitos
- **Normalização**: ajuda o treinamento do MLP.
- **Divisão treino/teste**: parte dos dados é reservada para avaliar o modelo.

## 3. Construção do Modelo

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential([
    Dense(64, activation='relu', input_dim=X_train.shape[1]),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

### Conceitos
- **Camadas e neurônios**: MLP com camadas densas.
- **Função de ativação**: usamos ReLU e softmax.
- **Dropout**: ajuda a reduzir overfitting.

## 4. Treinamento

In [ ]:
import tensorflow as tf
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)]
)

## 5. Avaliação

In [ ]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f'Acurácia de teste: {acc:.4f}')

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='val')
plt.xlabel('Epoch')
plt.ylabel('Acurácia')
plt.legend()
plt.show()

### Conceitos
- **Crossentropy** como função de custo.
- **Acurácia** como métrica de desempenho.
- **Overfitting** monitorado via validação.

## 6. Matriz de Confusão

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
y_pred = model.predict(X_test).argmax(axis=1)
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(cm)
disp.plot()
plt.show()